<a href="https://colab.research.google.com/github/ThanujSingaravelan/master_thesis_battery_uncertainty/blob/main/thesis_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!git clone https://github.com/ThanujSingaravelan/master_thesis_battery_uncertainty.git
%cd master_thesis_battery_uncertainty 

Cloning into 'master_thesis_battery_uncertainty'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 13 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.
/content/master_thesis_battery_uncertainty/master_thesis_battery_uncertainty


In [13]:
import pandas as pd
import uncertainty_modelling_core


In [ ]:
df_BNN = pd.read_csv('demo_data_BNN_model.csv', index_col=False)

uncertain_data = uncertainty_modelling_core.UncertaintyData()

# df_BNN.loc[6500:7000,'voltage'] = 350.0

df_demo_regr = df_BNN[0:7000]
uncertain_data.add_data(df_demo_regr)

uncertain_data.compute_tensors_windowed(stride=1, window_size=10, clustering_mode=False)
training_data_regr = uncertain_data.get_tensor_data()